# RAG using Enterprise Bing Search
Whilst DuckDuckGo was a nice and simple example to use with RAG you probably might not want to use it within your company. Especially if you already have an Azure account. I would advise to use Bing Search instead, which incidently is the API of choice for ChatGPT as well! Here is the code for the function and some instructions on how to deploy it within your Azure Resource Group.

Bing Search API Credentials
To utilize Bing's Search API:
1. **Sign Up**:
   - If you haven't already, sign up for an [Azure trial account](https://azure.com/free).
   
2. **Deploy Bing Search API**:
   - Navigate to "Create a Resource" > "AI + Machine Learning" > "Bing Search v7".
   - Fill out the form and deploy the resource.
3. **Retrieve Credentials**:
   - Once deployed, navigate to the "Keys and Endpoint" section of your Bing Search resource in Azure portal.
   - Copy one of the provided keys and the endpoint.
Add these to your `.env` file:
- BING_URL=your_bing_endpoint_here
- BING_KEY=your_bing_api_key_here

Just replace the RAG function with the one below:

In [ ]:
import os
import datetime
import requests
from gptrim import trim

def search_internet(query):
    """
    Use Bing Search API to provide context
    """
    endpoint = f"{os.getenv('BING_URL')}/v7.0/search"
    params = {
        'q': query,
        'count': 5,
        'responseFilter': 'webpages',
        'freshness': 'month'
    }
    current_date = datetime.date.today()
    past_month_start = current_date - datetime.timedelta(days=30)
    params['freshnessstartdate'] = past_month_start.strftime('%Y-%m-%d')
    params['freshnessenddate'] = current_date.strftime('%Y-%m-%d')
    headers = { 'Ocp-Apim-Subscription-Key': os.getenv('BING_KEY') }
    # Call the API
    response = requests.get(endpoint, headers=headers, params=params)
    
    if response.status_code == 200:
        response_json = response.json()
        search_results = response_json.get('webPages', {}).get('value', [])
        result_text = ''
        for result in search_results:
            title = result.get('name', '')
            snippet = result.get('snippet', '')
            url = result.get('url', '')
            result_text += f'Title: {title}\nSnippet: {snippet}\nURL: {url}\n\n'

        search_prompt = f"""
        Based on the internet search results provided in <>, provide an answer to the query [] if it is relevant along with a source URL. \
        If there are no internet search results or if they are not relevant then say \"Please try again.\"\
        
        context:<{trim(result_text)}>
        query:[{query}]
        """

        print ('Original text source:\n'+result_text)

        print ('Trimmed text source:\n'+trim(result_text))

        return {"role":"system","content": search_prompt}
    
    else:
        return {"role":"system", "content": "Say: 'Please try again.'"}

Change to normal strings for Vertex...

In [ ]:

import datetime
import requests
import os
from gptrim import trim

def search_internet(query):
    """
    Use Bing Search API to provide context
    """
    endpoint = f"{os.getenv('BING_URL')}/v7.0/search"
    params = {
        'q': query,
        'count': 5,
        'responseFilter': 'webpages',
        'freshness': 'month'
    }
    current_date = datetime.date.today()
    past_month_start = current_date - datetime.timedelta(days=30)
    params['freshnessstartdate'] = past_month_start.strftime('%Y-%m-%d')
    params['freshnessenddate'] = current_date.strftime('%Y-%m-%d')
    headers = { 'Ocp-Apim-Subscription-Key': os.getenv('BING_KEY') }
    # Call the API
    response = requests.get(endpoint, headers=headers, params=params)
    
    if response.status_code == 200:
        response_json = response.json()
        search_results = response_json.get('webPages', {}).get('value', [])
        result_text = ''
        for result in search_results:
            title = result.get('name', '')
            snippet = result.get('snippet', '')
            url = result.get('url', '')
            result_text += f'Title: {title}\nSnippet: {snippet}\nURL: {url}\n\n'

        search_prompt = f"""
        Based on the internet search results provided in <>, provide an answer to the query [] if it is relevant along with a source URL. \
        If there are no internet search results or if they are not relevant then say \"Please try again.\"\
        
        context:<{trim(result_text)}>
        query:[{query}]
        """

        print ('Original text source:\n'+result_text)

        print ('Trimmed text source:\n'+trim(result_text))

        return search_prompt
    
    else:
        return "Say: 'Please try again.'"